In [ ]:
import numpy as np
import os
import pandas as pd
import xlsxwriter
import csv
from datetime import datetime, timedelta

# ============ CONFIGURACIÓN - Actualizar estas variables según sea necesario ============
Fecha_Envio = "13-02-2026"
Año = "2026"
Mes_Carpeta = "02_Febrero"
Dia_Maestro = "13"

# Nombres de archivos de entrada
S2Contributivo = "S2 EPSC No valido.TXT"
S2Subsidiado = "S2EPS02510022026.TXT"
R2Contributivo = "R2EPSC2510022026.TXT"
R2Subsidiado = "R2EPS02510022026.TXT"

# Nombres de archivos de salida
S4Contributivo = "No Valido.TXT"
S4Subsidiado = "S4EPS02513022026.TXT"
R4Contributivo = "R4EPSC2513022026.TXT"
R4Subsidiado = "R4EPS02513022026.TXT"

# Nombres de hojas de Excel
nombre_hoja_MS_YESID = "Sheet1"
nombre_hoja_SISBEN = "Resultado"

# ============ CONSTRUCCIÓN AUTOMÁTICA DE RUTAS ============
# Detectar usuario actual de Windows
usuario_actual = os.environ.get('USERNAME', os.getlogin())

# Ruta base de OneDrive (igual para todos los usuarios)
onedrive_base = fr"C:\Users\{usuario_actual}\OneDrive - 891856000_CAPRESOCA E P S"

# Rutas base de trabajo
capresoca_base = os.path.join(onedrive_base, "Capresoca", "AlmostClear")
bdua_sub = os.path.join(capresoca_base, "Procesos BDUA", "Subsidiados", "Procesos BDUA EPS")
bdua_con = os.path.join(capresoca_base, "Procesos BDUA", "Contributivo", "Procesos BDUA EPS")
malla_adres = os.path.join(onedrive_base, "Documentos", "malla adres")

# Rutas de archivos de entrada (S2, R2)
Ruta_S2_Subsidiado = os.path.join(bdua_sub, "S2", Año, S2Subsidiado)
Ruta_S2_Contributivo = os.path.join(bdua_con, "S2", Año, S2Contributivo)
Ruta_R2_Subsidiado = os.path.join(bdua_sub, "R2", Año, R2Subsidiado)
Ruta_R2_Contributivo = os.path.join(bdua_con, "R2", Año, R2Contributivo)

# Rutas de archivos unificados
Ruta_S2S_Unificado = os.path.join(bdua_sub, "S2", "S2S_unificado.txt")
Ruta_S2C_Unificado = os.path.join(bdua_con, "S2", "S2C_unificado.txt")
Ruta_R2S_Unificado = os.path.join(bdua_sub, "R2", "R2S_unificado.txt")
Ruta_R2C_Unificado = os.path.join(bdua_con, "R2", "R2C_unificado.txt")

# Rutas de archivos auxiliares
Ruta_Consulta_Sisben = os.path.join(capresoca_base, "SISBEN", Año, Mes_Carpeta, "DNP-SISBEN-CO-0000086146.xlsx")
Ruta_Usuarios_Retirar = os.path.join(onedrive_base, "Escritorio", "Yesid Rincón Z", "Traslados", 
                                      "Procesos BDUA", Año, Mes_Carpeta, Dia_Maestro, "Maestro-retiros.xlsx")

# Rutas de archivos de salida (S4, R4)
InformeTraslado = os.path.join(malla_adres, f"{Fecha_Envio}-Respuestas_Traslados.xlsx")
Ruta_S4_Subsidiado = os.path.join(malla_adres, "Subsidiado", S4Subsidiado)
Ruta_S4_Contributivo = os.path.join(malla_adres, "Contributivo", S4Contributivo)
Ruta_R4_Subsidiado = os.path.join(malla_adres, "Subsidiado", R4Subsidiado)
Ruta_R4_Contributivo = os.path.join(malla_adres, "Contributivo", R4Contributivo)
    


# VALIDACIÓN DE RUTAS Y ARCHIVOS

In [ ]:
# ============ VALIDACIÓN DE RUTAS Y ARCHIVOS ============
print("=" * 80)
print("VALIDACIÓN DE CONFIGURACIÓN")
print("=" * 80)
print(f"\n📅 Fecha: {Fecha_Envio} | Año: {Año} | Mes: {Mes_Carpeta} | Día: {Dia_Maestro}")
print(f"👤 Usuario detectado: {usuario_actual}")
print(f"📂 OneDrive base: {onedrive_base}")

# Verificar si OneDrive existe
if os.path.exists(onedrive_base):
    print("✅ Ruta de OneDrive encontrada")
else:
    print("❌ ERROR: Ruta de OneDrive NO encontrada")

print("\n" + "-" * 80)
print("ARCHIVOS DE ENTRADA (deben existir)")
print("-" * 80)

archivos_entrada = {
    "S2 Subsidiado": Ruta_S2_Subsidiado,
    "S2 Contributivo": Ruta_S2_Contributivo,
    "R2 Subsidiado": Ruta_R2_Subsidiado,
    "R2 Contributivo": Ruta_R2_Contributivo,
    "S2S Unificado": Ruta_S2S_Unificado,
    "S2C Unificado": Ruta_S2C_Unificado,
    "R2S Unificado": Ruta_R2S_Unificado,
    "R2C Unificado": Ruta_R2C_Unificado,
    "SISBEN": Ruta_Consulta_Sisben,
    "Maestro Retiros": Ruta_Usuarios_Retirar
}

errores_entrada = []
for nombre, ruta in archivos_entrada.items():
    existe = os.path.exists(ruta)
    estado = "✅" if existe else "❌"
    print(f"{estado} {nombre:20s} | {ruta}")
    if not existe:
        errores_entrada.append(nombre)

print("\n" + "-" * 80)
print("CARPETAS DE SALIDA (se crearán si no existen)")
print("-" * 80)

carpetas_salida = {
    "Malla ADRES": malla_adres,
    "Malla ADRES/Subsidiado": os.path.dirname(Ruta_S4_Subsidiado),
    "Malla ADRES/Contributivo": os.path.dirname(Ruta_S4_Contributivo)
}

for nombre, ruta in carpetas_salida.items():
    existe = os.path.exists(ruta)
    if not existe:
        try:
            os.makedirs(ruta, exist_ok=True)
            print(f"📁 {nombre:30s} | Carpeta creada: {ruta}")
        except Exception as e:
            print(f"❌ {nombre:30s} | Error al crear: {e}")
    else:
        print(f"✅ {nombre:30s} | {ruta}")

print("\n" + "-" * 80)
print("ARCHIVOS DE SALIDA (se generarán)")
print("-" * 80)

archivos_salida = {
    "Informe Traslados": InformeTraslado,
    "S4 Subsidiado": Ruta_S4_Subsidiado,
    "S4 Contributivo": Ruta_S4_Contributivo,
    "R4 Subsidiado": Ruta_R4_Subsidiado,
    "R4 Contributivo": Ruta_R4_Contributivo
}

for nombre, ruta in archivos_salida.items():
    print(f"📄 {nombre:20s} | {ruta}")

print("\n" + "=" * 80)
if errores_entrada:
    print(f"⚠️  ADVERTENCIA: {len(errores_entrada)} archivo(s) de entrada NO encontrado(s):")
    for error in errores_entrada:
        print(f"   - {error}")
    print("\n❌ NO se puede continuar. Verifique los archivos faltantes.")
    print("=" * 80)
else:
    print("✅ VALIDACIÓN EXITOSA - Todos los archivos de entrada existen")
    print("=" * 80)
    print("\n🚀 Listo para ejecutar el procesamiento...")

# Ejecucion

In [ ]:
#####cargar los archivos en dataframes // encoding='cp1252' = a decodificar ANSI // header=None es sin encabezado
#Se carga el S2_subsidiado en un dataframe
Df_S2_Subsidiado = pd.read_csv(Ruta_S2_Subsidiado, delimiter=',', header=None, encoding='cp1252')
#Se carga el S2_Contributivo en un dataframe
Df_S2_Contributivo = pd.read_csv(Ruta_S2_Contributivo, delimiter=',', header=None, encoding='cp1252')
#Se carga el R2_subsidiado en un dataframe
Df_R2_Subsidiado = pd.read_csv(Ruta_R2_Subsidiado, delimiter=',', header=None, encoding='cp1252')
#Se carga el R2_Contributivo en un dataframe
Df_R2_Contributivo = pd.read_csv(Ruta_R2_Contributivo, delimiter=',', header=None, encoding='cp1252')
# Obtener la cantidad de columnas en el DataFrame
num_columnas_S2S = Df_S2_Subsidiado.shape[1]
num_columnas_S2C = Df_S2_Contributivo.shape[1]
num_columnas_R2S = Df_R2_Subsidiado.shape[1]
num_columnas_R2C = Df_R2_Contributivo.shape[1]
# Crear una lista de nombres de columnas
nombres_columnas_S2S = [f"Columna{i+1}" for i in range(num_columnas_S2S)]
nombres_columnas_S2C = [f"Columna{i+1}" for i in range(num_columnas_S2C)]
nombres_columnas_R2S = [f"Columna{i+1}" for i in range(num_columnas_R2S)]
nombres_columnas_R2C = [f"Columna{i+1}" for i in range(num_columnas_R2C)]
# Asignar los nombres de columnas al DataFrame
Df_S2_Subsidiado.columns = nombres_columnas_S2S
Df_S2_Contributivo.columns = nombres_columnas_S2C
Df_R2_Subsidiado.columns = nombres_columnas_R2S
Df_R2_Contributivo.columns = nombres_columnas_R2C


####### Se carga excel de consulta sisben  ######
# Cargar la hoja "Resultado" en un DataFrame
Df_Consulta_Sisben = pd.read_excel(Ruta_Consulta_Sisben, sheet_name=nombre_hoja_SISBEN)
####### Se carga excel Maestro-Yesid usuarios para retirar  ######
# Cargar la hoja "Resultado" en un DataFrame
Df_Usuarios_Retirar = pd.read_excel(Ruta_Usuarios_Retirar, sheet_name=nombre_hoja_MS_YESID)


def crear_nuevo_dataframe(input_df):
    # Crear un nuevo DataFrame con las primeras cuatro columnas del DataFrame de entrada
    nuevo_df = input_df.iloc[:, :4].copy()

    # Agregar las columnas vacías al nuevo DataFrame
    nuevo_df['Sisben'] = ''
    nuevo_df['motivo-aprovación'] = ''
    nuevo_df['Contar S2'] = ''
    nuevo_df['Contar R2'] = ''

    return nuevo_df

#Se agregan las 4 columnas al dataframe
Df_S4_Subsidiado = crear_nuevo_dataframe(Df_S2_Subsidiado)
Df_S4_Contributivo = crear_nuevo_dataframe(Df_S2_Contributivo)
Df_R4_Subsidiado = crear_nuevo_dataframe(Df_R2_Subsidiado)
Df_R4_Contributivo = crear_nuevo_dataframe(Df_R2_Contributivo)
# Guardar Df_S4_Subsidiado en un archivo .txt con decodificación ANSI
Df_S4_Subsidiado.to_csv(Ruta_S4_Subsidiado, sep=',', index=False, encoding='ANSI')
# Guardar Df_S4_Contributivo en un archivo .txt con decodificación ANSI
Df_S4_Contributivo.to_csv(Ruta_S4_Contributivo, sep=',', index=False, encoding='ANSI')
# Guardar Df_R4_Subsidiado en un archivo .txt con decodificación ANSI
Df_R4_Subsidiado.to_csv(Ruta_R4_Subsidiado, sep=',', index=False, encoding='ANSI')
# Guardar Df_R4_Contributivo en un archivo .txt con decodificación ANSI
Df_R4_Contributivo.to_csv(Ruta_R4_Contributivo, sep=',', index=False, encoding='ANSI')

##Culsultamos S4 con sisben para optener departamento de sisben
# Df_Consulta_Sisben Convertir las columnas enteras a cadenas y eliminar espacios en 'tipoDocumento'
Df_Consulta_Sisben['tipoDocumento'] = Df_Consulta_Sisben['tipoDocumento'].astype(str).str.strip()
Df_Consulta_Sisben['numeroDocumento'] = Df_Consulta_Sisben['numeroDocumento'].astype(str)
Df_Consulta_Sisben['departamento'] = Df_Consulta_Sisben['departamento'].astype(str)
# Df_S4_Subsidiado Convertir las columnas enteras a cadenas y eliminar espacios en 'tipoDocumento'
Df_S4_Subsidiado['Columna4'] = Df_S4_Subsidiado['Columna4'].astype(str)
Df_S4_Subsidiado['Columna3'] = Df_S4_Subsidiado['Columna3'].astype(str)
Df_S4_Subsidiado['Sisben'] = Df_S4_Subsidiado['Sisben'].astype(str)
# Df_S4_Contributivo Convertir las columnas enteras a cadenas y eliminar espacios en 'tipoDocumento'
Df_S4_Contributivo['Columna4'] = Df_S4_Contributivo['Columna4'].astype(str)
Df_S4_Contributivo['Columna3'] = Df_S4_Contributivo['Columna3'].astype(str)
Df_S4_Contributivo['Sisben'] = Df_S4_Contributivo['Sisben'].astype(str)
# Df_R4_Contributivo Convertir las columnas enteras a cadenas y eliminar espacios en 'tipoDocumento'
Df_R4_Contributivo['Columna3'] = Df_R4_Contributivo['Columna3'].astype(str)
Df_R4_Contributivo['Columna4'] = Df_R4_Contributivo['Columna4'].astype(str)
Df_R4_Contributivo['Sisben'] = Df_R4_Contributivo['Sisben'].astype(str)
# Df_S4_Subsidiado Convertir las columnas enteras a cadenas y eliminar espacios en 'tipoDocumento'
Df_R4_Subsidiado['Columna4'] = Df_R4_Subsidiado['Columna4'].astype(str)
Df_R4_Subsidiado['Columna3'] = Df_R4_Subsidiado['Columna3'].astype(str)
Df_R4_Subsidiado['Sisben'] = Df_S4_Subsidiado['Sisben'].astype(str)



#Función para garegar el departameto sisben a los S4 y R4
def fusionar_y_agregar_sisben_departamento(df_base, df_fuente, columna_fuente, columna_destino):
    # Realizar la fusión entre df_base y df_fuente
    merged_df = df_base.merge(df_fuente, left_on=[columna_destino], right_on=[columna_fuente], how='left')
    
    # Agregar la columna 'motivo-aprovación' al DataFrame base
    df_base['Sisben'] = merged_df['departamento']
    
    return df_base

#Usar la función fusionar_y_agregar_motivo_aprobacion para agregar el motivo de aprovación
Df_S4_Subsidiado = fusionar_y_agregar_sisben_departamento(Df_S4_Subsidiado, Df_Consulta_Sisben, 'numeroDocumento', 'Columna4')
Df_S4_Contributivo = fusionar_y_agregar_sisben_departamento(Df_S4_Contributivo, Df_Consulta_Sisben, 'numeroDocumento', 'Columna4')


# Guardar Df_S4_Subsidiado en un archivo .txt con decodificación ANSI
Df_S4_Subsidiado.to_csv(Ruta_S4_Subsidiado, sep=',', index=False, encoding='ANSI')
# Guardar Df_S4_Contributivo en un archivo .txt con decodificación ANSI
Df_S4_Contributivo.to_csv(Ruta_S4_Contributivo, sep=',', index=False, encoding='ANSI')



##Culsultamos S4 con Maestro-Yesid.xlsx para optener motivo-aprovación.
# Convertir las columnas enteras a cadenas y eliminar espacios en 'tipoDocumento'
Df_Usuarios_Retirar['tipoDocumento'] = Df_Usuarios_Retirar['tipo_documento'].astype(str).str.strip()
Df_Usuarios_Retirar['numero_documento'] = Df_Usuarios_Retirar['numero_documento'].astype(str)
Df_Usuarios_Retirar['Restiros-yesid-S4-R4'] = Df_Usuarios_Retirar['Restiros-yesid-S4-R4'].astype(str)

#Función para garegar el motivo de aprovación a los S4 y R4
def fusionar_y_agregar_motivo_aprobacion(df_base, df_fuente, columna_fuente, columna_destino):
    # Realizar la fusión entre df_base y df_fuente
    merged_df = df_base.merge(df_fuente, left_on=[columna_destino], right_on=[columna_fuente], how='left')
    
    # Agregar la columna 'motivo-aprovación' al DataFrame base
    df_base['motivo-aprovación'] = merged_df['Restiros-yesid-S4-R4']
    
    return df_base

#Usar la función fusionar_y_agregar_motivo_aprobacion para agregar el motivo de aprovación
Df_S4_Subsidiado = fusionar_y_agregar_motivo_aprobacion(Df_S4_Subsidiado, Df_Usuarios_Retirar, 'numero_documento', 'Columna4')
Df_S4_Contributivo = fusionar_y_agregar_motivo_aprobacion(Df_S4_Contributivo, Df_Usuarios_Retirar, 'numero_documento', 'Columna4')
Df_R4_Subsidiado = fusionar_y_agregar_motivo_aprobacion(Df_R4_Subsidiado, Df_Usuarios_Retirar, 'numero_documento', 'Columna4')
Df_R4_Contributivo = fusionar_y_agregar_motivo_aprobacion(Df_R4_Contributivo, Df_Usuarios_Retirar, 'numero_documento', 'Columna4')


##Culsultamos S4 con sisben para optener departamento de sisben
# Df_Consulta_Sisben Convertir las columnas enteras a cadenas y eliminar espacios en 'tipoDocumento'
Df_Consulta_Sisben['tipoDocumento'] = Df_Consulta_Sisben['tipoDocumento'].astype(str).str.strip()
Df_Consulta_Sisben['numeroDocumento'] = Df_Consulta_Sisben['numeroDocumento'].astype(str)
Df_Consulta_Sisben['departamento'] = Df_Consulta_Sisben['departamento'].astype(str)
# Df_S4_Subsidiado Convertir las columnas enteras a cadenas y eliminar espacios en 'tipoDocumento'
Df_S4_Subsidiado['Columna4'] = Df_S4_Subsidiado['Columna4'].astype(str)
Df_S4_Subsidiado['Columna3'] = Df_S4_Subsidiado['Columna3'].astype(str)
Df_S4_Subsidiado['Sisben'] = Df_S4_Subsidiado['Sisben'].astype(str)
# Df_S4_Contributivo Convertir las columnas enteras a cadenas y eliminar espacios en 'tipoDocumento'
Df_S4_Contributivo['Columna4'] = Df_S4_Contributivo['Columna4'].astype(str)
Df_S4_Contributivo['Columna3'] = Df_S4_Contributivo['Columna3'].astype(str)
Df_S4_Contributivo['Sisben'] = Df_S4_Contributivo['Sisben'].astype(str)
# Df_R4_Contributivo Convertir las columnas enteras a cadenas y eliminar espacios en 'tipoDocumento'
Df_R4_Contributivo['Columna3'] = Df_R4_Contributivo['Columna3'].astype(str)
Df_R4_Contributivo['Columna4'] = Df_R4_Contributivo['Columna4'].astype(str)
Df_R4_Contributivo['Sisben'] = Df_R4_Contributivo['Sisben'].astype(str)
# Df_R4_Subsidiado Convertir las columnas enteras a cadenas y eliminar espacios en 'tipoDocumento'
Df_R4_Subsidiado['Columna4'] = Df_R4_Subsidiado['Columna4'].astype(str)
Df_R4_Subsidiado['Columna3'] = Df_R4_Subsidiado['Columna3'].astype(str)
Df_R4_Subsidiado['Sisben'] = Df_R4_Subsidiado['Sisben'].astype(str)


Df_Usuarios_Retirar['numero_documento'] = Df_Usuarios_Retirar['numero_documento'].astype(str).str.strip()
Df_S4_Subsidiado['Columna4'] = Df_S4_Subsidiado['Columna4'].astype(str).str.strip()
Df_S4_Contributivo['Columna4'] = Df_S4_Contributivo['Columna4'].astype(str).str.strip()
Df_R4_Subsidiado['Columna4'] = Df_R4_Subsidiado['Columna4'].astype(str).str.strip()
Df_R4_Contributivo['Columna4'] = Df_R4_Contributivo['Columna4'].astype(str).str.strip()

#Función para garegar el motivo de aprovación a los S4 y R4
def fusionar_y_agregar_motivo_aprobacion(df_base, df_fuente, columna_fuente, columna_destino):
    merged_df = df_base.merge(
        df_fuente,
        left_on=[columna_destino],
        right_on=[columna_fuente],
        how='left',
        suffixes=('', '_drop')
    )
    # Elimina columnas duplicadas del merge
    merged_df = merged_df[[col for col in merged_df.columns if not col.endswith('_drop')]]
    merged_df['motivo-aprovación'] = merged_df['Restiros-yesid-S4-R4'].fillna('')
    return merged_df

#Usar la función fusionar_y_agregar_motivo_aprobacion para agregar el motivo de aprovación
Df_S4_Subsidiado = fusionar_y_agregar_sisben_departamento(Df_S4_Subsidiado, Df_Consulta_Sisben, 'numeroDocumento', 'Columna4')
Df_S4_Contributivo = fusionar_y_agregar_sisben_departamento(Df_S4_Contributivo, Df_Consulta_Sisben, 'numeroDocumento', 'Columna4')
Df_R4_Subsidiado = fusionar_y_agregar_sisben_departamento(Df_R4_Subsidiado, Df_Consulta_Sisben, 'numeroDocumento', 'Columna4')
Df_R4_Contributivo = fusionar_y_agregar_sisben_departamento(Df_R4_Contributivo, Df_Consulta_Sisben, 'numeroDocumento', 'Columna4')

def cargar_archivo_Columnas_variable(ruta_archivo):
    data = []
    with open(ruta_archivo, 'r') as archivo:
        lector = csv.reader(archivo)
        for fila in lector:
            data.append(fila)
    
    max_columnas = max(len(fila) for fila in data)
    for fila in data:
        while len(fila) < max_columnas:
            fila.append(None)
    
    column_names = [f'columna{i}' for i in range(1, max_columnas + 1)]
    df = pd.DataFrame(data, columns=column_names)
    return df

##### Se carga el S2S_Unificado para contar cuantas veces ha sido solicitados #####
Df_S2S_Unificado = cargar_archivo_Columnas_variable(Ruta_S2S_Unificado)
##### Se carga el S2C_Unificado para contar cuantas veces ha sido solicitados #####
Df_S2C_Unificado = cargar_archivo_Columnas_variable(Ruta_S2C_Unificado)
##### Se carga el R2S_Unificado para contar cuantas veces ha sido solicitados #####
Df_R2S_Unificado = cargar_archivo_Columnas_variable(Ruta_R2S_Unificado)
##### Se carga el R2C_Unificado para contar cuantas veces ha sido solicitados #####
Df_R2C_Unificado = cargar_archivo_Columnas_variable(Ruta_R2C_Unificado)

##### unir los DataFrame S2 subsidiado y contributivo #####
Df_S2_Unificado = pd.concat([Df_S2S_Unificado, Df_S2C_Unificado], axis=0, ignore_index=True)
##### unir los DataFrame R2 subsidiado y contributivo #####
Df_R2_Unificado = pd.concat([Df_R2S_Unificado, Df_R2C_Unificado], axis=0, ignore_index=True)

#### Filtrar los datos según los criterios para eliminar movilidades y dejar solo solicitudes de traslado ###
condiciones_filtro_Movilidades_S = (
    (Df_S2_Unificado['columna3'] == 'EPS025') | (Df_S2_Unificado['columna3'] == 'EPSC25')
)
#### Filtrar los datos según los criterios para eliminar movilidades y dejar solo solicitudes de traslado ###
condiciones_filtro_Movilidades_C = (
    (Df_R2_Unificado['columna3'] == 'EPS025') | (Df_R2_Unificado['columna3'] == 'EPSC25')
)
# Eliminar los registros que cumplen las condiciones
Df_S2_Unificado = Df_S2_Unificado[~condiciones_filtro_Movilidades_S]
Df_R2_Unificado = Df_R2_Unificado[~condiciones_filtro_Movilidades_C]

##### Agrupar y contar los registros en Df_S2_Unificado #####
#S4 Subsidiado
grouped_counts_S2S = Df_S2_Unificado.groupby(['columna4', 'columna5']).size().reset_index(name='Contar')
# Combinar los resultados en Df_S4_Subsidiado usando merge
merged_df = Df_S4_Subsidiado.merge(grouped_counts_S2S, left_on=['Columna3', 'Columna4'], right_on=['columna4', 'columna5'], how='left')
# Llenar registros con cero donde no hubo coincidencias en Df_S2_Unificado
merged_df['Contar S2'] = merged_df['Contar'].fillna(0).astype(int)
Df_S4_Subsidiado['Contar S2'] = merged_df['Contar S2']

# S4 Contributivo
grouped_counts_S2C = Df_S2_Unificado.groupby(['columna4', 'columna5']).size().reset_index(name='Contar')
# Combinar los resultados en Df_S4_Contributivo usando merge
merged_df = Df_S4_Contributivo.merge(grouped_counts_S2C, left_on=['Columna3', 'Columna4'], right_on=['columna4', 'columna5'], how='left')
# Llenar registros con cero donde no hubo coincidencias en Df_S2_Unificado
merged_df['Contar S2'] = merged_df['Contar'].fillna(0).astype(int)
Df_S4_Contributivo['Contar S2'] = merged_df['Contar S2']

#R4 Subsidiado
grouped_counts_R2S = Df_S2_Unificado.groupby(['columna4', 'columna5']).size().reset_index(name='Contar')
# Combinar los resultados en Df_R4_Subsidiado usando merge
merged_df = Df_R4_Subsidiado.merge(grouped_counts_R2S, left_on=['Columna3', 'Columna4'], right_on=['columna4', 'columna5'], how='left')
# Llenar registros con cero donde no hubo coincidencias en Df_S2_Unificado
merged_df['Contar S2'] = merged_df['Contar'].fillna(0).astype(int)
Df_R4_Subsidiado['Contar S2'] = merged_df['Contar S2']

# R4 Contributivo
grouped_counts_R2C = Df_S2_Unificado.groupby(['columna4', 'columna5']).size().reset_index(name='Contar')
# Combinar los resultados en Df_R4_Contributivo usando merge
merged_df = Df_R4_Contributivo.merge(grouped_counts_R2C, left_on=['Columna3', 'Columna4'], right_on=['columna4', 'columna5'], how='left')
# Llenar registros con cero donde no hubo coincidencias en Df_S2_Unificado
merged_df['Contar S2'] = merged_df['Contar'].fillna(0).astype(int)
Df_R4_Contributivo['Contar S2'] = merged_df['Contar S2']


##### Agrupar y contar los registros en Df_R2_Unificado #####
# S4 Subsidiado
grouped_counts_S2S = Df_R2_Unificado.groupby(['columna4', 'columna5']).size().reset_index(name='Contar')
# Combinar los resultados en Df_S4_Subsidiado usando merge
merged_df = Df_S4_Subsidiado.merge(grouped_counts_S2S, left_on=['Columna3', 'Columna4'], right_on=['columna4', 'columna5'], how='left')
# Llenar registros con cero donde no hubo coincidencias en Df_R2_Unificado
merged_df['Contar R2'] = merged_df['Contar'].fillna(0).astype(int)
Df_S4_Subsidiado['Contar R2'] = merged_df['Contar R2']

# S4 Contributivo
grouped_counts_S2C = Df_R2_Unificado.groupby(['columna4', 'columna5']).size().reset_index(name='Contar')
# Combinar los resultados en Df_S4_Subsidiado usando merge
merged_df = Df_S4_Contributivo.merge(grouped_counts_S2C, left_on=['Columna3', 'Columna4'], right_on=['columna4', 'columna5'], how='left')
# Llenar registros con cero donde no hubo coincidencias en Df_R2_Unificado
merged_df['Contar R2'] = merged_df['Contar'].fillna(0).astype(int)
Df_S4_Contributivo['Contar R2'] = merged_df['Contar R2']

# R4 Subsidiado
grouped_counts_R2S = Df_R2_Unificado.groupby(['columna4', 'columna5']).size().reset_index(name='Contar')
# Combinar los resultados en Df_R4_Subsidiado usando merge
merged_df = Df_R4_Subsidiado.merge(grouped_counts_R2S, left_on=['Columna3', 'Columna4'], right_on=['columna4', 'columna5'], how='left')
# Llenar registros con cero donde no hubo coincidencias en Df_R2_Unificado
merged_df['Contar R2'] = merged_df['Contar'].fillna(0).astype(int)
Df_R4_Subsidiado['Contar R2'] = merged_df['Contar R2']

# R4 Contributivo
grouped_counts_R2C = Df_R2_Unificado.groupby(['columna4', 'columna5']).size().reset_index(name='Contar')
# Combinar los resultados en Df_R4_Subsidiado usando merge
merged_df = Df_R4_Contributivo.merge(grouped_counts_R2C, left_on=['Columna3', 'Columna4'], right_on=['columna4', 'columna5'], how='left')
# Llenar registros con cero donde no hubo coincidencias en Df_R2_Unificado
merged_df['Contar R2'] = merged_df['Contar'].fillna(0).astype(int)
Df_R4_Contributivo['Contar R2'] = merged_df['Contar R2']

# Definir una función para calcular la siguiente fecha del primer día del mes
def siguiente_primer_dia_del_mes(fecha):
    siguiente_mes = fecha.replace(day=1) + timedelta(days=32)
    primer_dia_siguiente_mes = siguiente_mes.replace(day=1)
    return primer_dia_siguiente_mes.strftime('%d/%m/%Y')

# Filtrar filas donde 'motivo-aprovación' no sea vacío o nulo Df_S4_Subsidiado
filtro = Df_S4_Subsidiado['motivo-aprovación'].notna() & (Df_S4_Subsidiado['motivo-aprovación'] != '')
# Filas con motivo de aprobación no vacío o nulo
filas_con_motivo = Df_S4_Subsidiado[filtro]
# Obtener la fecha actual
fecha_actual = datetime.now()
# Llenar las columnas Columna5, Columna6 y Columna7
Df_S4_Subsidiado.loc[filtro, 'Columna5'] = 1
Df_S4_Subsidiado.loc[filtro, 'Columna6'] = 1
Df_S4_Subsidiado.loc[filtro, 'Columna7'] = siguiente_primer_dia_del_mes(fecha_actual)

# Filtrar filas donde 'motivo-aprovación' no sea vacío o nulo Df_S4_Contributivo
filtro = Df_S4_Contributivo['motivo-aprovación'].notna() & (Df_S4_Contributivo['motivo-aprovación'] != '')
# Filas con motivo de aprobación no vacío o nulo
filas_con_motivo = Df_S4_Contributivo[filtro]
# Obtener la fecha actual
fecha_actual = datetime.now()
# Llenar las columnas Columna5, Columna6 y Columna7
Df_S4_Contributivo.loc[filtro, 'Columna5'] = 1
Df_S4_Contributivo.loc[filtro, 'Columna6'] = 1
Df_S4_Contributivo.loc[filtro, 'Columna7'] = siguiente_primer_dia_del_mes(fecha_actual)

# Filtrar filas donde 'motivo-aprovación' no sea vacío o nulo Df_R4_Subsidiado
filtro = Df_R4_Subsidiado['motivo-aprovación'].notna() & (Df_R4_Subsidiado['motivo-aprovación'] != '')
# Filas con motivo de aprobación no vacío o nulo
filas_con_motivo = Df_R4_Subsidiado[filtro]
# Obtener la fecha actual
fecha_actual = datetime.now()
# Llenar las columnas Columna5, Columna6 y Columna7
Df_R4_Subsidiado.loc[filtro, 'Columna5'] = 1
Df_R4_Subsidiado.loc[filtro, 'Columna6'] = 1
Df_R4_Subsidiado.loc[filtro, 'Columna7'] = siguiente_primer_dia_del_mes(fecha_actual)

# Filtrar filas donde 'motivo-aprovación' no sea vacío o nulo Df_R4_Contributivo
filtro = Df_R4_Contributivo['motivo-aprovación'].notna() & (Df_R4_Contributivo['motivo-aprovación'] != '')
# Filas con motivo de aprobación no vacío o nulo
filas_con_motivo = Df_R4_Contributivo[filtro]
# Obtener la fecha actual
fecha_actual = datetime.now()
# Llenar las columnas Columna5, Columna6 y Columna7
Df_R4_Contributivo.loc[filtro, 'Columna6'] = 1
Df_R4_Contributivo.loc[filtro, 'Columna5'] = 1
Df_R4_Contributivo.loc[filtro, 'Columna7'] = siguiente_primer_dia_del_mes(fecha_actual)

# Definir una función para calcular la siguiente fecha del primer día del mes
def siguiente_primer_dia_del_mes(fecha):
    siguiente_mes = fecha.replace(day=1) + timedelta(days=62) # 31 días (mes actual) + 31 días (siguiente mes)
    primer_dia_siguiente_mes = siguiente_mes.replace(day=1)
    return primer_dia_siguiente_mes.strftime('%d/%m/%Y')

# Filtrar filas donde Sisben no sea vacío, nulo o 'CASANARE' Df_S4_Subsidiado
filtro = (
    ((Df_S4_Subsidiado['Sisben'] != 'CASANARE') & (Df_S4_Subsidiado['Sisben'].notna()) & (Df_S4_Subsidiado['Sisben'] != '')) &
    ((Df_S4_Subsidiado['Sisben'] == 'Columna5') | (Df_S4_Subsidiado['Columna5'].isna()))
)
# Filas con Sisben no vacío, nulo o 'CASANARE'
filas_con_sisben = Df_S4_Subsidiado[filtro]
# Obtener la fecha actual
fecha_actual = datetime.now()
# Llenar las columnas Columna5, Columna6 y Columna7
Df_S4_Subsidiado.loc[filtro, 'Columna5'] = 1
Df_S4_Subsidiado.loc[filtro,'Columna6'] = 4
#Df_S4_Subsidiado.loc[filtro,'Columna7'] = siguiente_primer_dia_del_mes(fecha_actual)


# Filtrar filas donde Sisben no sea vacío, nulo o 'CASANARE' Df_S4_Contributivo
filtro = (
    ((Df_S4_Contributivo['Sisben'] != 'CASANARE') & (Df_S4_Contributivo['Sisben'].notna()) & (Df_S4_Contributivo['Sisben'] != '')) &
    ((Df_S4_Contributivo['Sisben'] == 'Columna5') | (Df_S4_Contributivo['Columna5'].isna()))
)
# Filas con Sisben no vacío, nulo o 'CASANARE'
filas_con_sisben = Df_S4_Contributivo[filtro]
# Obtener la fecha actual
fecha_actual = datetime.now()
# Llenar las columnas Columna5, Columna6 y Columna7
Df_S4_Contributivo.loc[filtro, 'Columna5'] = 1
Df_S4_Contributivo.loc[filtro,'Columna6'] = 4
#Df_S4_Contributivo.loc[filtro,'Columna7'] = siguiente_primer_dia_del_mes(fecha_actual)

# Filtrar filas donde Sisben no sea vacío, nulo o 'CASANARE' Df_R4_Subsidiado
filtro = (
    ((Df_R4_Subsidiado['Sisben'] != 'CASANARE') & (Df_R4_Subsidiado['Sisben'].notna()) & (Df_R4_Subsidiado['Sisben'] != '')) &
    ((Df_R4_Subsidiado['Sisben'] == 'Columna5') | (Df_R4_Subsidiado['Columna5'].isna()))
)
# Filas con Sisben no vacío, nulo o 'CASANARE'
filas_con_sisben = Df_R4_Subsidiado[filtro]
# Obtener la fecha actual
fecha_actual = datetime.now()
# Llenar las columnas Columna5, Columna6 y Columna7
Df_R4_Subsidiado.loc[filtro, 'Columna5'] = 1
Df_R4_Subsidiado.loc[filtro,'Columna6'] = 4
#Df_R4_Subsidiado.loc[filtro,'Columna7'] = siguiente_primer_dia_del_mes(fecha_actual)

# Filtrar filas donde Sisben no sea vacío, nulo o 'CASANARE' Df_R4_Contributivo
filtro = (
    ((Df_R4_Contributivo['Sisben'] != 'CASANARE') & (Df_R4_Contributivo['Sisben'].notna()) & (Df_R4_Contributivo['Sisben'] != '')) &
    ((Df_R4_Contributivo['Sisben'] == 'Columna5') | (Df_R4_Contributivo['Columna5'].isna()))
)
# Filas con Sisben no vacío, nulo o 'CASANARE'
filas_con_sisben = Df_R4_Contributivo[filtro]
# Obtener la fecha actual
fecha_actual = datetime.now()
# Llenar las columnas Columna5, Columna6 y Columna7
Df_R4_Contributivo.loc[filtro, 'Columna5'] = 1
Df_R4_Contributivo.loc[filtro,'Columna6'] = 4
#Df_R4_Contributivo.loc[filtro,'Columna7'] = siguiente_primer_dia_del_mes(fecha_actual)


# Definir una función para calcular la siguiente fecha del primer día del mes
def siguiente_primer_dia_del_mes(fecha):
    siguiente_mes = fecha.replace(day=1) + timedelta(days=32)
    primer_dia_siguiente_mes = siguiente_mes.replace(day=1)
    return primer_dia_siguiente_mes.strftime('%d/%m/%Y')


# Convertir las columnas 'Contar S2' y 'Contar R2' a enteros Df_S4_Subsidiado
Df_S4_Subsidiado['Contar S2'] = pd.to_numeric(Df_S4_Subsidiado['Contar S2'], errors='coerce').fillna(0).astype(int)
Df_S4_Subsidiado['Contar R2'] = pd.to_numeric(Df_S4_Subsidiado['Contar R2'], errors='coerce').fillna(0).astype(int)
# Filtro para las filas donde 'Columna5' es vacío o nulo
filtro_vacio_nulo = Df_S4_Subsidiado['Columna5'].isna() | (Df_S4_Subsidiado['Columna5'] == '')
# Filtro para las filas donde la suma de 'Contar S2' y 'Contar R2' es mayor que 20
filtro_suma_mas_de_5 = (Df_S4_Subsidiado['Contar S2'] + Df_S4_Subsidiado['Contar R2']) > 20
# Aplicar ambos filtros
filtro_final = filtro_vacio_nulo & filtro_suma_mas_de_5
# Obtener la fecha actual
fecha_actual = datetime.now()
# Llenar las columnas Columna5, Columna6 y Columna7
Df_S4_Subsidiado.loc[filtro_final, 'Columna5'] = 1
Df_S4_Subsidiado.loc[filtro_final, 'Columna6'] = 1
#Df_S4_Subsidiado.loc[filtro_final, 'Columna7'] = siguiente_primer_dia_del_mes(fecha_actual)


# Convertir las columnas 'Contar S2' y 'Contar R2' a enteros Df_S4_Contributivo
Df_S4_Contributivo['Contar S2'] = pd.to_numeric(Df_S4_Contributivo['Contar S2'], errors='coerce').fillna(0).astype(int)
Df_S4_Contributivo['Contar R2'] = pd.to_numeric(Df_S4_Contributivo['Contar R2'], errors='coerce').fillna(0).astype(int)
# Filtro para las filas donde 'Columna5' es vacío o nulo
filtro_vacio_nulo = Df_S4_Contributivo['Columna5'].isna() | (Df_S4_Contributivo['Columna5'] == '')
# Filtro para las filas donde la suma de 'Contar S2' y 'Contar R2' es mayor que 20
filtro_suma_mas_de_5 = (Df_S4_Contributivo['Contar S2'] + Df_S4_Contributivo['Contar R2']) > 20
# Aplicar ambos filtros
filtro_final = filtro_vacio_nulo & filtro_suma_mas_de_5
# Obtener la fecha actual
fecha_actual = datetime.now()
# Llenar las columnas Columna5, Columna6 y Columna7
Df_S4_Contributivo.loc[filtro_final, 'Columna5'] = 1
Df_S4_Contributivo.loc[filtro_final, 'Columna6'] = 1
#Df_S4_Contributivo.loc[filtro_final, 'Columna7'] = siguiente_primer_dia_del_mes(fecha_actual)

# Convertir las columnas 'Contar S2' y 'Contar R2' a enteros Df_R4_Subsidiado
Df_R4_Subsidiado['Contar S2'] = pd.to_numeric(Df_R4_Subsidiado['Contar S2'], errors='coerce').fillna(0).astype(int)
Df_R4_Subsidiado['Contar R2'] = pd.to_numeric(Df_R4_Subsidiado['Contar R2'], errors='coerce').fillna(0).astype(int)
# Filtro para las filas donde 'Columna5' es vacío o nulo
filtro_vacio_nulo = Df_R4_Subsidiado['Columna5'].isna() | (Df_R4_Subsidiado['Columna5'] == '')
# Filtro para las filas donde la suma de 'Contar S2' y 'Contar R2' es mayor que 20
filtro_suma_mas_de_5 = (Df_R4_Subsidiado['Contar S2'] + Df_R4_Subsidiado['Contar R2']) > 20
# Aplicar ambos filtros
filtro_final = filtro_vacio_nulo & filtro_suma_mas_de_5
# Obtener la fecha actual
fecha_actual = datetime.now()
# Llenar las columnas Columna5, Columna6 y Columna7
Df_R4_Subsidiado.loc[filtro_final, 'Columna5'] = 1
Df_R4_Subsidiado.loc[filtro_final, 'Columna6'] = 1
#Df_R4_Subsidiado.loc[filtro_final, 'Columna7'] = siguiente_primer_dia_del_mes(fecha_actual)

# Convertir las columnas 'Contar S2' y 'Contar R2' a enteros Df_R4_Contributivo
Df_R4_Contributivo['Contar R2'] = pd.to_numeric(Df_R4_Contributivo['Contar R2'], errors='coerce').fillna(0).astype(int)
Df_R4_Contributivo['Contar S2'] = pd.to_numeric(Df_R4_Contributivo['Contar S2'], errors='coerce').fillna(0).astype(int)
# Filtro para las filas donde 'Columna5' es vacío o nulo
filtro_vacio_nulo = Df_R4_Contributivo['Columna5'].isna() | (Df_R4_Contributivo['Columna5'] == '')
# Filtro para las filas donde la suma de 'Contar S2' y 'Contar R2' es mayor que 20
filtro_suma_mas_de_5 = (Df_R4_Contributivo['Contar S2'] + Df_R4_Contributivo['Contar R2']) > 20
# Aplicar ambos filtros
filtro_final = filtro_vacio_nulo & filtro_suma_mas_de_5
# Obtener la fecha actual
fecha_actual = datetime.now()
# Llenar las columnas Columna5, Columna6 y Columna7
Df_R4_Contributivo.loc[filtro_final, 'Columna5'] = 1
Df_R4_Contributivo.loc[filtro_final, 'Columna6'] = 1
#Df_R4_Contributivo.loc[filtro_final, 'Columna7'] = siguiente_primer_dia_del_mes(fecha_actual)


# Filtro para las filas donde 'Columna5' es vacío o nulo Df_S4_Subsidiado
filtro_vacio_nulo = Df_S4_Subsidiado['Columna5'].isna() | (Df_S4_Subsidiado['Columna5'] == '')
# Obtener la fecha actual
fecha_actual = datetime.now()
# Llenar las columnas Columna5, Columna6 y Columna7
Df_S4_Subsidiado.loc[filtro_vacio_nulo, 'Columna5'] = 0
Df_S4_Subsidiado.loc[filtro_vacio_nulo, 'Columna6'] = 10


# Filtro para las filas donde 'Columna5' es vacío o nulo Df_S4_Contributivo
filtro_vacio_nulo = Df_S4_Contributivo['Columna5'].isna() | (Df_S4_Contributivo['Columna5'] == '')
# Obtener la fecha actual
fecha_actual = datetime.now()
# Llenar las columnas Columna5, Columna6 y Columna7
Df_S4_Contributivo.loc[filtro_vacio_nulo, 'Columna5'] = 0
Df_S4_Contributivo.loc[filtro_vacio_nulo, 'Columna6'] = 10
 
# Filtro para las filas donde 'Columna5' es vacío o nulo Df_R4_Subsidiado
filtro_vacio_nulo = Df_R4_Subsidiado['Columna5'].isna() | (Df_R4_Subsidiado['Columna5'] == '')
# Obtener la fecha actual
fecha_actual = datetime.now()
# Llenar las columnas Columna5, Columna6 y Columna7
Df_R4_Subsidiado.loc[filtro_vacio_nulo, 'Columna5'] = 0
Df_R4_Subsidiado.loc[filtro_vacio_nulo, 'Columna6'] = 10

# Filtro para las filas donde 'Columna5' es vacío o nulo Df_R4_Contributivo
filtro_vacio_nulo = Df_R4_Contributivo['Columna5'].isna() | (Df_R4_Contributivo['Columna5'] == '')
# Obtener la fecha actual
fecha_actual = datetime.now()
# Llenar las columnas Columna5, Columna6 y Columna7
Df_R4_Contributivo.loc[filtro_vacio_nulo, 'Columna5'] = 0
Df_R4_Contributivo.loc[filtro_vacio_nulo, 'Columna6'] = 10

Df_S4_Subsidiado['Columna5'] = Df_S4_Subsidiado['Columna5'].astype(int)
Df_S4_Subsidiado['Columna6'] = Df_S4_Subsidiado['Columna6'].astype(int)
Df_S4_Contributivo['Columna5'] = Df_S4_Contributivo['Columna5'].astype(int)
Df_S4_Contributivo['Columna6'] = Df_S4_Contributivo['Columna6'].astype(int)
Df_R4_Contributivo['Columna5'] = Df_R4_Contributivo['Columna5'].astype(int)
Df_R4_Contributivo['Columna6'] = Df_R4_Contributivo['Columna6'].astype(int)
Df_R4_Subsidiado['Columna5'] = Df_R4_Subsidiado['Columna5'].astype(int)
Df_R4_Subsidiado['Columna6'] = Df_R4_Subsidiado['Columna6'].astype(int)


# Crear S4_Subsidiado con un subconjunto de columnas en el orden deseado
S4_Subsidiado = Df_S4_Subsidiado[['Columna1', 'Columna2', 'Columna3', 'Columna4', 'Columna5', 'Columna6', 'Columna7']]
# Guardar S4_Subsidiado en un archivo CSV
S4_Subsidiado.to_csv(Ruta_S4_Subsidiado, sep=',', index=False, encoding='ANSI', header=False)                                                                                                                                                                                                                                                                                                                               
# Crear Respuestas_Traslados con todas las columnas en el orden deseado
column_order = ['Columna1', 'Columna2', 'Columna3', 'Columna4', 'Columna5', 'Columna6', 'Columna7', 'Sisben', 'motivo-aprovación', 'Contar S2', 'Contar R2']
Respuestas_Traslados_S4S = Df_S4_Subsidiado[column_order]

# Crear S4_Contributivo con un subconjunto de columnas en el orden deseado
S4_Contributivo = Df_S4_Contributivo[['Columna1', 'Columna2', 'Columna3', 'Columna4', 'Columna5', 'Columna6', 'Columna7']]
# Guardar S4_Contributivo en un archivo CSV
S4_Contributivo.to_csv(Ruta_S4_Contributivo, sep=',', index=False, encoding='ANSI', header=False)                                                                                                                                                                                                                                                                                                                               
# Crear Respuestas_Traslados con todas las columnas en el orden deseado
column_order = ['Columna1', 'Columna2', 'Columna3', 'Columna4', 'Columna5', 'Columna6', 'Columna7', 'Sisben', 'motivo-aprovación', 'Contar S2', 'Contar R2']
Respuestas_Traslados_S4C = Df_S4_Contributivo[column_order]

# Crear una nueva columna llamada 'indice' con los valores formateados Df_S4_Contributivo
Df_R4_Subsidiado['indice'] = (Df_R4_Subsidiado.index + 1).astype(str).str.zfill(5)
# Crear R4_Subsidiado con un subconjunto de columnas en el orden deseado
R4_Subsidiado = Df_R4_Subsidiado[['Columna1', 'Columna2', 'Columna3', 'Columna4', 'indice', 'Columna5', 'Columna6', 'Columna7']]
# Guardar R4_Subsidiado en un archivo CSV
R4_Subsidiado.to_csv(Ruta_R4_Subsidiado, sep=',', index=False, encoding='ANSI', header=False)                                                                                                                                                                                                                                                                                                                               
# Crear Respuestas_Traslados con todas las columnas en el orden deseado
column_order = ['Columna1', 'Columna2', 'Columna3', 'Columna4', 'indice', 'Columna5', 'Columna6', 'Columna7', 'Sisben', 'motivo-aprovación', 'Contar S2', 'Contar R2']
Respuestas_Traslados_R4S = Df_R4_Subsidiado[column_order]

# Crear una nueva columna llamada 'indice' con los valores formateados Df_S4_Contributivo
Df_R4_Contributivo['indice'] = (Df_R4_Contributivo.index + 1).astype(str).str.zfill(5)
# Crear R4_Contributivo con un subconjunto de columnas en el orden deseado
R4_Contributivo = Df_R4_Contributivo[['Columna1', 'Columna2', 'Columna3', 'Columna4', 'indice', 'Columna5', 'Columna6', 'Columna7']]
# Guardar R4_Contributivo en un archivo CSV
R4_Contributivo.to_csv(Ruta_R4_Contributivo, sep=',', index=False, encoding='ANSI', header=False)                                                                                                                                                                                                                                                                                                                               
# Crear Respuestas_Traslados con todas las columnas en el orden deseado
column_order = ['Columna1', 'Columna2', 'Columna3', 'Columna4', 'indice', 'Columna5', 'Columna6', 'Columna7', 'Sisben', 'motivo-aprovación', 'Contar S2', 'Contar R2']
Respuestas_Traslados_R4C = Df_R4_Contributivo[column_order]

#Crea Dataframe Resumen de los archivos generados
columnas = ['Nombre_Archivo', 'EPS025', 'EPSC25', 'Aprovados_EPS025', 'Aprovados_EPSC25', 'Negados_EPS025', 'Negados_EPSC25']
df_vacio = pd.DataFrame(columns=columnas)
df_vacio.loc[0, 'Nombre_Archivo'] = 'R4'
df_vacio.loc[1, 'Nombre_Archivo'] = 'S4'
# Contar la cantidad de registros (filas) en R4_Subsidiado
cantidad_registros = len(R4_Subsidiado)
# Asignar la cantidad de registros
df_vacio.loc[0, 'EPS025'] = cantidad_registros
cantidad_registros = len(R4_Contributivo)
df_vacio.loc[0, 'EPSC25'] = cantidad_registros
cantidad_registros = len(S4_Subsidiado)
df_vacio.loc[1, 'EPS025'] = cantidad_registros
cantidad_registros = len(S4_Contributivo)
df_vacio.loc[1, 'EPSC25'] = cantidad_registros
# Contar la cantidad de registros donde Columna5 es igual a 1
cantidad_columna5_igual_1 = len(R4_Subsidiado[R4_Subsidiado['Columna5'] == 1])
# Asignar esta cantidad 1 de la Aprovados
df_vacio.loc[0, 'Aprovados_EPS025'] = cantidad_columna5_igual_1
cantidad_columna5_igual_1 = len(R4_Contributivo[R4_Contributivo['Columna5'] == 1])
df_vacio.loc[0, 'Aprovados_EPSC25'] = cantidad_columna5_igual_1
cantidad_columna5_igual_1 = len(S4_Subsidiado[S4_Subsidiado['Columna5'] == 1])
df_vacio.loc[1, 'Aprovados_EPS025'] = cantidad_columna5_igual_1
cantidad_columna5_igual_1 = len(S4_Contributivo[S4_Contributivo['Columna5'] == 1])
df_vacio.loc[1, 'Aprovados_EPSC25'] = cantidad_columna5_igual_1
# Asignar esta cantidad 0 de la Negados
cantidad_columna5_igual_1 = len(R4_Subsidiado[R4_Subsidiado['Columna5'] == 0])
df_vacio.loc[0, 'Negados_EPS025'] = cantidad_columna5_igual_1
cantidad_columna5_igual_1 = len(R4_Contributivo[R4_Contributivo['Columna5'] == 0])
df_vacio.loc[0, 'Negados_EPSC25'] = cantidad_columna5_igual_1
cantidad_columna5_igual_1 = len(S4_Subsidiado[S4_Subsidiado['Columna5'] == 0])
df_vacio.loc[1, 'Negados_EPS025'] = cantidad_columna5_igual_1
cantidad_columna5_igual_1 = len(S4_Contributivo[S4_Contributivo['Columna5'] == 0])
df_vacio.loc[1, 'Negados_EPSC25'] = cantidad_columna5_igual_1



# Crear un nuevo archivo Excel
writer = pd.ExcelWriter(InformeTraslado, engine='xlsxwriter')
# Escribir Resumen en la hoja 'Resumen' de los archivo Excel
df_vacio.to_excel(writer, sheet_name='Resumen', index=False)
# Escribir Respuestas_Traslados en la hoja 'S2-Contributivo' del archivo Excel
Df_S2_Contributivo.to_excel(writer, sheet_name='S2-contributivo', index=False)
# Escribir Respuestas_Traslados en la hoja 'S4-Contributivo' del archivo Excel
Respuestas_Traslados_S4C.to_excel(writer, sheet_name='S4-contributivo', index=False)
# Escribir Respuestas_Traslados en la hoja 'S2-Subsidiado' del archivo Excel
Df_S2_Subsidiado.to_excel(writer, sheet_name='S2-subsidiado', index=False)
# Escribir Respuestas_Traslados en la hoja 'S4-subsidiado' del archivo Excel
Respuestas_Traslados_S4S.to_excel(writer, sheet_name='S4-subsidiado', index=False)
# Escribir Respuestas_Traslados en la hoja 'R2-Contributivo' del archivo Excel
Df_R2_Contributivo.to_excel(writer, sheet_name='R2-contributivo', index=False)
# Escribir Respuestas_Traslados en la hoja 'R4-Contributivo' del archivo Excel
Respuestas_Traslados_R4C.to_excel(writer, sheet_name='R4-contributivo', index=False)
# Escribir Respuestas_Traslados en la hoja 'R2-Subsidiado' del archivo Excel
Df_R2_Subsidiado.to_excel(writer, sheet_name='R2-subsidiado', index=False)
# Escribir Respuestas_Traslados en la hoja 'R4-subsidiado' del archivo Excel
Respuestas_Traslados_R4S.to_excel(writer, sheet_name='R4-subsidiado', index=False)
# Guardar el archivo Excels
writer.close()